In [ ]:
# !pip install cvxpy
!pip install --upgrade --pre cvxpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.3.3
    Uninstalling cvxpy-1.3.3:
      Successfully uninstalled cvxpy-1.3.3


In [ ]:
# !pip install gurobipy
!pip install mosek

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 47.4 MB/s eta 0:00:00


In [ ]:
import cvxpy as cp
# import gurobipy as gp
import mosek
import numpy as np
from sklearn.datasets import load_breast_cancer,load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestCentroid
from scipy.spatial.distance import cdist


In [ ]:
np.random.seed(1)
sc=StandardScaler()

In [ ]:
df=pd.read_csv('/content/diabetes.csv')

In [ ]:
df

,Age,Gender,BMI,BloodPressure,TCells,LowDensityLipoproteins,HighDensityLipoproteins,ThyroidStimulatingHormone,Lamotrigine,BloodSugarLevel,DiseaseProgression
0,59,1,32.1,101.00,157,93.2,38.0,4.00,4.8598,87,1
1,48,0,21.6,87.00,183,103.2,70.0,3.00,3.8918,69,0
2,72,1,30.5,93.00,156,93.6,41.0,4.00,4.6728,85,1
3,24,0,25.3,84.00,198,131.4,40.0,5.00,4.8903,89,1
4,50,0,23.0,101.00,192,125.4,52.0,4.00,4.2905,80,0
...,...,...,...,...,...,...,...,...,...,...,...
437,60,1,28.2,112.00,185,113.8,42.0,4.00,4.9836,93,1
438,47,1,24.9,75.00,225,166.0,42.0,5.00,4.4427,102,0
439,60,1,24.9,99.67,162,106.6,43.0,3.77,4.1271,95,0
440,36,0,30.0,95.00,201,125.2,42.0,4.79,5.1299,85,1


In [ ]:
df=pd.read_csv('/content/diabetes.csv')
dataset = df #.loc[2:,:]
# dataset.info()
# dataset[['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']]=dataset[['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']].astype('int')
# dataset[['Class']]=dataset[['Class']].astype('int')

In [ ]:
def preprocess(numerical,categorical):
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    transformations = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical),
            ('cat', categorical_transformer, categorical)])
    return transformations

In [ ]:
y = dataset["DiseaseProgression"]
# Split data into train and test
X = dataset.drop("DiseaseProgression", axis=1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42) #rs 0,1 pinv also doesnt work

In [ ]:
numerical = ['Age','BMI','BloodPressure','TCells','LowDensityLipoproteins','HighDensityLipoproteins','ThyroidStimulatingHormone','Lamotrigine','BloodSugarLevel']
categorical = x_train.columns.difference(numerical)
proc=preprocess(numerical,categorical)
x_train=proc.fit_transform(x_train)
x_test=proc.transform(x_test)
y_train=y_train.values
y_test=y_test.values

In [ ]:
clf = LogisticRegression(random_state=0).fit(x_train,y_train)

In [ ]:
# clf=MLPClassifier(random_state=0,hidden_layer_sizes=(10,),activation='identity').fit(x_train,y_train)

In [ ]:
def create_obj_var(x,n_ft,ft_type):
  vars=[]
  obj_var=[0]*n_ft
  for i in range(n_ft):
    # vars[i]=cp.Variable()
    # print(ft_type[i])
    if ft_type[i]=='ct':
      # print(vars[i])
      v=cp.Variable()
      obj_var[i]=cp.norm(v-x[i],1)/4 #should add denominator
    elif ft_type[i]=='in':
      v=cp.Variable(integer=True)
      obj_var[i]=cp.norm(v-x[i],1)/4
    else:
      v=cp.Variable(boolean=True)
      obj_var[i]=(cp.maximum(1,(v-x[i])/1000))
    vars.append(v)
  # vars=cp.Variable(n_ft)
  return vars,obj_var



In [ ]:
def create_obj_var2(x,n_ft,ft_type,A,c,target):
  vars=[]
  nu=0.5
  obj_var=[0]*n_ft
  if target==0:
    A=A[0]
    c=c[0]
  else:
    A=A[1]
    c=c[1]
  for i in range(n_ft):
    # vars[i]=cp.Variable()
    # print(ft_type[i])
    if ft_type[i]=='ct':
      # print(vars[i])
      v=cp.Variable()
      obj_var[i]=cp.norm(v-x[i],1)/4 #should add denominator
    elif ft_type[i]=='in':
      v=cp.Variable(integer=True)
      obj_var[i]=cp.norm(v-x[i],1)/4
    else:
      v=cp.Variable(boolean=True)
      obj_var[i]=(cp.maximum(1,(v-x[i])/1000))

    vars.append(v)

  for i in range(n_ft):
    obj_var[i]=obj_var[i]++nu*cp.norm(A[i,:]@(vars-c),1)
  # vars=cp.Variable(n_ft)
  return vars,obj_var



In [ ]:
# vars,_=create_obj_var(x_test[0],30,['ct']*30)
# vars

In [ ]:
# p=vars@clf.coef_[0].T
# cp.exp(p)/1+cp.exp(p)

In [ ]:
# np.logaddexp(-1000,x_test[0]@clf.coef_[0]+clf.intercept_[0])

In [ ]:
# p=x_test[0]@clf.coef_[0]+clf.intercept_[0]
# (np.exp(p)/(1+np.exp(p)))
# p

In [ ]:
def check_pred(cf,clf,target,clf_type='linear'): #assume mlp activation is identity
  if clf_type=='linear':
    cf=cp.hstack(cf)
    # p=cp.log_sum_exp(cp.hstack([-1000,cf@clf.coef_[0].T+clf.intercept_[0]]))
    pr=cp.matmul(cf,clf.coef_[0])+clf.intercept_[0]
    # p=cp.exp(p)/(1+cp.exp(p))
    p=cp.sum(cp.exp(cp.abs(pr - cp.log(target))))

  elif clf_type=='mlp':
    pr=cp.hstack(cf)
    for i in range(len(clf.coefs_)):
      pr=cp.matmul(pr,clf.coefs_[i])+clf.intercepts_[i]

    p=cp.sum(cp.exp(cp.abs(pr - cp.log(target))))
  return p,pr



In [ ]:
def get_anchors(x_train,y_train):
  cls1_idx=np.where(y_train==1)
  cls0_idx=np.where(y_train==0)

  cls0=x_train[cls0_idx]
  cls1=x_train[cls1_idx]

  cls0_var=np.linalg.pinv(np.cov(cls0.T))
  cls1_var=np.linalg.pinv(np.cov(cls1.T))

  clf = NearestCentroid()
  clf.fit(x_train,y_train)

  return cls0_var,cls1_var,clf.centroids_



In [ ]:
# import cvxpy as cp
# import numpy as np
# from sklearn.linear_model import LogisticRegression

# # Generate some example data
# np.random.seed(0)  # For reproducibility
# X = np.random.rand(100, 2)  # Example input features
# y = np.random.randint(0, 2, 100)  # Example binary labels (0 or 1)

# # Fit a logistic regression model
# logreg_model = LogisticRegression()
# logreg_model.fit(X, y)

# # Define cvxpy variable for the input vector
# x = cp.Variable(2)

# # Compute the log odds (linear part of logistic regression)
# log_odds = cp.matmul(x, logreg_model.coef_[0]) + logreg_model.intercept_

# # Define the target probability (e.g., close to zero)
# target_probability = 0.3  # Adjust as needed

# # Define the log-sum-exp constraint indirectly
# constraint_lhs = cp.sum(cp.exp(log_odds - cp.log(target_probability)))
# constraint_rhs = len(X)  # Number of data points
# constraint = constraint_lhs <= constraint_rhs

# # Create the cvxpy problem with the constraint
# problem = cp.Problem(cp.Minimize(0), [constraint])

# # Solve the problem
# problem.solve()

# # Access the optimized input vector
# optimal_vector = x.value

# # Evaluate the log odds for the optimal vector
# optimal_log_odds = np.dot(optimal_vector, logreg_model.coef_[0]) + logreg_model.intercept_

# print("Optimal Input Vector:", optimal_vector)
# print("Log Odds for Optimal Vector:", optimal_log_odds)


In [ ]:
# Create constraint.
def optimize(vars,obj_var,clf,x,target):
  # vars.append(cp.Variable())
  # x=x.reshape(1,-1)

  # p=cp.sum(vars*np.transpose(np.array(clf.coef_))) +clf.intercept_[0]
  # print(p)
  # p=1/1+cp.exp(-p)


  p,pr=check_pred(vars,clf,target,'mlp')
  constraints=[p<=1]
  # constraints=[]

  # cls0_var,cls1_var,centroids=get_anchors(x_train,y_train)
  # c1=centroids[1]
  # c0=centroids[0]
  # if target==0:
  #   # obj2=cp.matmul((cp.vstack(vars)-cp.vstack(c0)).T,cp.matmul(cls0_var,(cp.vstack(vars)-cp.vstack(c0))))
  #   obj2=cp.norm(cls0_var@(vars-c0),1)
  # else:
  #   obj2=cp.norm(cls1_var@(vars-c1),1)
  #   # obj2=cp.matmul((cp.vstack(vars)-cp.vstack(c1)).T,cp.matmul(cls1_var,(cp.vstack(vars)-cp.vstack(c1))))
  # Form objective.
  # objective = cp.Minimize(cp.sqrt(cp.sum_squares(cp.vstack(obj_var)))) #
  objective = cp.Minimize(cp.sum(cp.vstack(obj_var)))
  # objective = cp.Minimize(cp.norm(v-x,1))

  # vars.value = numpy.random.randn(30)


  # Form and solve problem.
  prob = cp.Problem(objective,constraints)
  prob.solve(solver=cp.MOSEK,qcp=True,verbose=True) #cp.SCS, cp.ECOS_BB is infeasible_inaccurate
  # prob.solve(warm_start=True)
  print("status: {}".format(prob.status))
  return vars

In [ ]:
print(cp.installed_solvers())

['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


In [ ]:
# cf_all=[]
# ft_type=['ct']*30
# n_ft=30
# for k in range(len(x_test)):
#   i=x_test[k]
#   target=1-y_test[k]
#   cls0_var,cls1_var,centroids=get_anchors(x_train,y_train)
#   vars,obj_var=create_obj_var2(i,n_ft,ft_type,[cls0_var,cls1_var],centroids,target)
#   # vars,obj_var=create_obj_var(i,n_ft,ft_type)
#   cf=[]
#   target=0.001 if target==0 else 0.999
#   vals=optimize(vars,obj_var,clf,i,target)
#   for j in vals:
#     cf.append(j.value)
#   cf_all.append(cf)

In [ ]:
np.array(cf_all).shape, y_test.shape

((0,), (500,))

In [ ]:
def check_validity(cf_all,y_test):
  validity=0
  for i in range(len(cf_all)):
    if clf.predict(np.array(cf_all[i]).reshape(1,-1))==1-y_test[i]:
      validity+=1

  return validity/len(y_test)

In [ ]:
def l1_dist(cf,x_test):
  d=[]
  for i in range(len(x_test)):
    d.append(cdist([x_test[0]],[cf[i]],metric='cityblock'))

  return np.mean(d)


In [ ]:
def maha_dist(cf,x_test,cov):
  d=[]

  for i in range(len(x_test)):
    diff=cf[i]-x_test[i]
    d.append(np.dot(diff.T,np.dot(cov,diff)))
  return np.mean(d)

In [ ]:
dist=l1_dist(cf_all,x_test)

NameError: name 'cf_all' is not defined

In [ ]:
mdist=maha_dist(cf_all,x_test,np.linalg.pinv(np.cov(x_test.T)))

In [ ]:
def get_stats(x_train,ft_types):
  '''
  should receive the raw data
  '''
  n_ft= x_train.shape[1]
  stats=[]
  for i in range(n_ft):
    s=[]
    if ft_types[i]=='ct':
      s[0]=np.mean(x_train[:,i])
      s[1]=np.var(x_train[:,i])**0.5
    else: #consider only cont and cat currently. Need discrete data also.
      s[0]=np.min(x_train[:,i])
      s[1]=np.max(x_train[:,i])
    stats.append(s)

  return stats


In [ ]:
def gen_random_point(stats,ft_types):
  randx=[]
  for i in range(len(stats)):
    val=0
    if ft_types[i]=='continuous':
      val=np.randn(stats[i][0],stats[i][1],size=None)
    else:
      val=np.random.randint(stats[i][0],stats[i][1])
    randx[i]=val

  return randx



5 fold

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
import time

In [ ]:
X=X.reset_index(drop=True)
y=y.values

In [ ]:
!mkdir /root/mosek

In [ ]:
!mv /content/mosek.lic /root/mosek/


In [ ]:
l1_dist_all=[]
maha_dist_all=[]
time_all=[]
valid_all=[]

for _, (train_index, test_index) in enumerate(kf.split(X)):
  x_train,x_test=X.iloc[train_index,:],X.iloc[test_index,:]
  y_train,y_test=y[train_index],y[test_index]
  numerical = ['Age','Gender','BMI','BloodPressure','TCells','LowDensityLipoproteins','HighDensityLipoproteins','ThyroidStimulatingHormone','Lamotrigine','BloodSugarLevel']
  categorical = x_train.columns.difference(numerical)
  proc=preprocess(numerical,categorical)
  x_train=proc.fit_transform(x_train)
  x_test=proc.transform(x_test)
  # y_train=y_train.values
  # y_test=y_test.values
  # clf = LogisticRegression(random_state=0).fit(x_train,y_train)
  clf=MLPClassifier(random_state=0,hidden_layer_sizes=(10,3),activation='identity').fit(x_train,y_train)
  cf_all=[]
  ft_type=['in']+['bb']+['ct']*2+['in']+['ct']*4+['in']
  n_ft=10
  start=time.time()
  for k in range(len(x_test)):
    i=x_test[k]
    target=1-y_test[k]
    cls0_var,cls1_var,centroids=get_anchors(x_train,y_train)
    vars,obj_var=create_obj_var2(i,n_ft,ft_type,[cls0_var,cls1_var],centroids,target)
    # vars,obj_var=create_obj_var(i,n_ft,ft_type)
    cf=[]
    target=0.001 if target==0 else 0.999
    vals=optimize(vars,obj_var,clf,i,target)
    for j in vals:
      cf.append(j.value)
    cf_all.append(cf)

  elapsed=time.time()-start


  l1_dist_all.append(l1_dist(cf_all,x_test))
  maha_dist_all.append(maha_dist(cf_all,x_test,np.linalg.pinv(np.cov(x_test.T))))
  valid_all.append(check_validity(cf_all,y_test))
  time_all.append(elapsed/len(x_test))


In [ ]:
print('avg l1 {}'.format(np.mean(l1_dist_all)))
print('avg maha {}'.format(np.mean(maha_dist_all)))
print('avg valid {}'.format(np.mean(valid_all)))
print('avg time {}'.format(np.mean(time_all)))

avg l1 14.025407282420138
avg maha 31.728058241808487
avg valid 0.5000510725229826
avg time 0.4603844933478167


In [ ]:
acc_all=[]
from sklearn.metrics import accuracy_score
for _, (train_index, test_index) in enumerate(kf.split(X)):
  x_train,x_test=X.iloc[train_index,:],X.iloc[test_index,:]
  y_train,y_test=y[train_index],y[test_index]
  numerical = ['Age', 'Duration','Job', 'SavingAccounts','CheckingAccount','CreditAmount']
  categorical = x_train.columns.difference(numerical)
  proc=preprocess(numerical,categorical)
  x_train=proc.fit_transform(x_train)
  x_test=proc.transform(x_test)
  # y_train=y_train.values
  # y_test=y_test.values
  # clf = LogisticRegression(random_state=0).fit(x_train,y_train)
  clf=MLPClassifier(random_state=0,hidden_layer_sizes=(10,3),activation='identity').fit(x_train,y_train)
  acc_all.append(accuracy_score(y_test,clf.predict(x_test)))

In [ ]:
print('avg acc {}'.format(np.mean(acc_all)))

avg acc 0.696
